<a href="https://www.kaggle.com/code/jonasaacampos/uso-de-express-es-regulares-no-tratamento-de-dados?scriptVersionId=121364810" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

[![](https://img.shields.io/badge/feito%20com%20%E2%9D%A4%20por-jaac-cyan)](https://github.com/jonasaacampos)
[![LinkedIn Badge](https://img.shields.io/badge/LinkedIn-Profile-informational?style=flat&logo=linkedin&logoColor=white&color=0D76A8)](https://www.linkedin.com/in/jonasaacampos)

[//]: # (Title: Uso de expressões regulares para Limpeza de dados: Funções anônimas e Pandas)
[//]: # (Author: Jonas Campos)
[//]: # (Date: March 07, 2023)
[//]: # (Comment: Projeto de análise de comparação entre métodos nativos do pandas e lambda functions) 
[//]: # (Tags: #python, #pandas, #lambda_functions, #map, #regex)  

# Uso de expressões regulares para Limpeza de dados: Funções anônimas e Pandas


In [ ]:
# biblioteca necessária para gerar dados aleatórios para teste

!pip install faker

In [ ]:
# biblioteca para geração de dados aleatórios
from faker import Faker

# biblioteca para usarmos Regular Expression 'regex' com o pyuthon
import re

# biblioteca de análise e manipulação de dados
import pandas as pd

In [ ]:
# criando nosso gerador e determinando o número de nossa amostra. Neste exemplo faremos a geração de 10 mil endereços fictícios
faker = Faker()
n = 10_000


In [ ]:
# função de preenche um conjunto de dados com dados aleatórios

def preenche_dataset():
    return pd.Series([faker.address() for i in range (n)])

In [ ]:
# gerando um conjunto de dados
df_enderecos = preenche_dataset()

In [ ]:
#amostra dos dados
df_enderecos

Nested dados, queremos extrair os dados: 

- **Estado:** sigla de duas letras após a vírgula, e separadas por um espaço do código postal
- **Código Postal:** código numérico no final de cada linha

Seguindo este padrão, (_PW 69878_), vamos considerar o trecho de cada linha que conhenha

- [ ] 2 letras
- [ ] 1 espaço
- [ ] 5 números

Em Regex, este padrão é escrito como `\w{2} \d{5}`

In [ ]:
# definindo o padrão da expressão regular

pattern = "\w{2} \d{5}"

## Caso 1: Funções anônima e expressão regular (lambda functions and regex)

O map() é uma função que executa determinada ação em cada item de um objeto. No nosso caso, para cada linha de endereço, vamos:

1. procurar pelo nosso padrão (2 letras + espaço + 5 números)
1. separar este trecho encontrado por meio do espaçamento entre as letras e números
1. renomear as colunas para e inserir cada dado em sua respectiva coluna


In [ ]:
(
    (df_enderecos)
        .map (lambda x: (re.search(pattern, x).group()))
        .str .split(" ", expand=True)
        .rename(columns={0: "state", 1:"zip_code"})    
)

## Caso 2: Pandas e regex de forma nativa



Separamos nosso padrão de análise textual, e agora vamos fazer a extração de grupos nomeados em cada parão, utilizando o método `extract()` nativo do pandas.

In [ ]:
df_enderecos.str.extract( r"(?P<state>\w{2}) (?P<zip_code>\d{5})" ) 

## Comparação entre as possibilidades

```python
# lambda function
(
    (df_enderecos)
        .map (lambda x: (re.search(pattern, x).group()))
        .str .split(" ", expand=True)
        .rename(columns={0: "state", 1:"zip_code"})    
)

# Pandas
df_enderecos.str.extract( r"(?P<state>\w{2}) (?P<zip_code>\d{5})" ) 

```

Ambos os métodos trazem o mesmo resultado, todavia estilo do pandas é mais _pythônico_ e mais simples. **Todavia** este método é um pouco mais lento, mas nada que inviabilize o seu uso.

No próximo artigo farei uma análise do tempo de execução de ambos os cenários.

## Cont(r)ate-me 📫:

<p align='center'>
  <a href='https://github.com/jonasaacampos'>
    <img src='https://img.shields.io/badge/GitHub-100000?style=for-the-badge&logo=github&logoColor=white'/>
  </a>
  <a href='https://www.linkedin.com/in/jonasaacampos/'>
    <img src='https://img.shields.io/badge/LinkedIn-0077B5?style=for-the-badge&logo=linkedin&logoColor=white'/>
  </a>
   <a href='https://dev.to/jonasaacampos'>
    <img src='https://img.shields.io/badge/dev.to-0A0A0A?style=for-the-badge&logo=devdotto&logoColor=white'/>
  </a>
    <a href='https://www.buymeacoffee.com/jaac.dev'>
    <img src='https://img.shields.io/badge/Buy_Me_A_Coffee-FFDD00?style=for-the-badge&logo=buy-me-a-coffee&logoColor=black'/>
  </a>
</p>
